# Decision Trees

In [193]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## 1. Decision Tree Regression

In [194]:
# TODO: Write a regression tree class that can build tree and make prediction
class RegressionTree:
    def __init__(self):
        self.tree = None
        self._cur_depth = 0

    def mse_impurity_measure(self, y):
        if len(y) == 0:
            return 0
        mean_y = y.mean()
        return ((y - mean_y) ** 2).mean()

    def find_best_th(self, X, y):
        X_sorted = X.sort_values()
        y_sorted = y[X_sorted.index]

        unique_vals = X.unique()
        if unique_vals.size < 2:
            return None, np.inf

        best_th = None
        best_impurity = np.inf
        thresholds = (unique_vals[:-1] + unique_vals[1:]) / 2

        for th in thresholds:
            y_left = y_sorted[X_sorted <= th]
            y_right = y_sorted[X_sorted > th]

            if len(y_left) == 0 or len(y_right) == 0:
                continue

            left_impurity = self.mse_impurity_measure(y_left)
            right_impurity = self.mse_impurity_measure(y_right)

            weighted_impurity = left_impurity * (
                y_left.shape[0] / y.shape[0]
            ) + right_impurity * (y_right.shape[0] / y.shape[0])

            if weighted_impurity < best_impurity:
                best_impurity = weighted_impurity
                best_th = th

        return best_th, best_impurity

    def find_best_feature(self, X, y):
        best_feature = None
        best_th = None
        best_impurity = np.inf

        for feature in X.columns:
            th, impurity = self.find_best_th(X[feature], y)

            if impurity < best_impurity:
                best_th = th
                best_impurity = impurity
                best_feature = feature

        return best_feature, best_th, best_impurity

    def fit(self, X, y, min_samples=5, max_depth=3):
        if len(X) < min_samples or self._cur_depth >= max_depth:
            return {"type": "leaf", "prediction": y.mean(), "depth": self._cur_depth}

        # Find the best feature and threshold to split on
        best_feature, best_th, best_impurity = self.find_best_feature(X, y)

        if best_feature is None:
            return {"type": "leaf", "prediction": y.mean(), "depth": self._cur_depth}

        X_left = X[X[best_feature] <= best_th]
        X_right = X[X[best_feature] > best_th]

        y_left = y[X_left.index]
        y_right = y[X_right.index]

        self._cur_depth += 1

        left_subtree = self.fit(
            X=X_left,
            y=y_left,
            min_samples=min_samples,
            max_depth=max_depth,
        )
        right_subtree = self.fit(
            X=X_right,
            y=y_right,
            min_samples=min_samples,
            max_depth=max_depth,
        )
        self._cur_depth -= 1

        self.tree = {
            "type": "node",
            "feature": best_feature,
            "threshold": best_th,
            "left": left_subtree,
            "right": right_subtree,
            "depth": self._cur_depth,
        }

        return {
            "type": "node",
            "feature": best_feature,
            "threshold": best_th,
            "left": left_subtree,
            "right": right_subtree,
            "depth": self._cur_depth,
        }

    def predict(self, X):
        current_node = self.tree

        while True:
            if current_node["type"] == "leaf":
                return current_node["prediction"]

            feature = current_node["feature"]
            threshold = current_node["threshold"]

            if X[feature] <= threshold:
                current_node = current_node["left"]
            else:
                current_node = current_node["right"]

In [195]:
df = pd.read_csv("../data/study_score.csv", index_col=False)
df

,StudyHours,SleepHours,CoffeeCups,Score,Pass
0,1,9,0,3,0
1,2,8,1,5,0
2,3,7,1,8,1
3,4,7,2,12,1
4,5,6,2,15,1
5,6,6,3,14,1
6,7,5,3,10,1
7,8,5,4,7,0
8,9,4,4,4,0
9,10,4,5,2,0


In [196]:
X = df.drop(columns=["Pass", "Score"], axis=1)
y = df["Score"]
X, y

(   StudyHours  SleepHours  CoffeeCups
 0           1           9           0
 1           2           8           1
 2           3           7           1
 3           4           7           2
 4           5           6           2
 5           6           6           3
 6           7           5           3
 7           8           5           4
 8           9           4           4
 9          10           4           5,
 0     3
 1     5
 2     8
 3    12
 4    15
 5    14
 6    10
 7     7
 8     4
 9     2
 Name: Score, dtype: int64)

In [197]:
tree = RegressionTree()

In [198]:
tree.fit(X, y)

{'type': 'node',
 'feature': 'StudyHours',
 'threshold': np.float64(8.5),
 'left': {'type': 'node',
  'feature': 'StudyHours',
  'threshold': np.float64(3.5),
  'left': {'type': 'leaf',
   'prediction': np.float64(5.333333333333333),
   'depth': 2},
  'right': {'type': 'node',
   'feature': 'StudyHours',
   'threshold': np.float64(6.5),
   'left': {'type': 'leaf',
    'prediction': np.float64(13.666666666666666),
    'depth': 3},
   'right': {'type': 'leaf', 'prediction': np.float64(8.5), 'depth': 3},
   'depth': 2},
  'depth': 1},
 'right': {'type': 'leaf', 'prediction': np.float64(3.0), 'depth': 1},
 'depth': 0}

In [199]:
tree.predict(X={"CoffeeCups": 3, "SleepHours": 6, "StudyHours": 10})

np.float64(3.0)

## 2. Decision Classification

In [200]:
# TODO: Write a classificatin tree class that can build tree and make prediction
class ClassificationTree:
    def __init__(self):
        self.tree = None
        self._cur_depth = 0

    def gini_impurity_measure(self, y):
        if len(y) == 0:
            return 0

        proportions = y.value_counts(normalize=True)
        return 1 - np.sum(proportions**2)

    def find_best_th(self, X, y):
        X_sorted = X.sort_values()
        y_sorted = y[X_sorted.index]

        unique_vals = X.unique()
        if unique_vals.size < 2:
            return None, np.inf

        best_th = None
        best_impurity = np.inf
        thresholds = (unique_vals[:-1] + unique_vals[1:]) / 2

        for th in thresholds:
            y_left = y_sorted[X_sorted <= th]
            y_right = y_sorted[X_sorted > th]

            if len(y_left) == 0 or len(y_right) == 0:
                continue

            left_impurity = self.gini_impurity_measure(y_left)
            right_impurity = self.gini_impurity_measure(y_right)

            weighted_impurity = (y_left.shape[0] / y.shape[0]) * left_impurity + (
                y_right.shape[0] / y.shape[0]
            ) * right_impurity

            if weighted_impurity < best_impurity:
                best_impurity = weighted_impurity
                best_th = th

        return best_th, best_impurity

    def find_best_feature(self, X, y):
        best_feature = None
        best_th = None
        best_impurity = np.inf

        for feature in X.columns:
            th, impurity = self.find_best_th(X[feature], y)

            if impurity < best_impurity:
                best_th = th
                best_impurity = impurity
                best_feature = feature

        return best_feature, best_th, best_impurity

    def fit(self, X, y, min_samples=5, max_depth=3):
        if len(X) < min_samples or self._cur_depth >= max_depth:
            return {"type": "leaf", "prediction": y.mode()[0], "depth": self._cur_depth}

        # Find the best feature and threshold to split on
        best_feature, best_th, best_var = self.find_best_feature(X, y)

        if best_feature is None:
            return {"type": "leaf", "prediction": y.mode()[0], "depth": self._cur_depth}

        X_left = X[X[best_feature] <= best_th]
        X_right = X[X[best_feature] > best_th]

        y_left = y[X_left.index]
        y_right = y[X_right.index]

        self._cur_depth += 1

        left_subtree = self.fit(
            X=X_left,
            y=y_left,
            min_samples=min_samples,
            max_depth=max_depth,
        )
        right_subtree = self.fit(
            X=X_right,
            y=y_right,
            min_samples=min_samples,
            max_depth=max_depth,
        )
        self._cur_depth -= 1

        self.tree = {
            "type": "node",
            "feature": best_feature,
            "threshold": best_th,
            "left": left_subtree,
            "right": right_subtree,
            "depth": self._cur_depth,
        }

        return {
            "type": "node",
            "feature": best_feature,
            "threshold": best_th,
            "left": left_subtree,
            "right": right_subtree,
            "depth": self._cur_depth,
        }

    def predict(self, X):
        current_node = self.tree

        while True:
            if current_node["type"] == "leaf":
                return current_node["prediction"]

            feature = current_node["feature"]
            threshold = current_node["threshold"]

            if X[feature] <= threshold:
                current_node = current_node["left"]
            else:
                current_node = current_node["right"]

In [201]:
df = pd.read_csv("../data/study_score.csv", index_col=False)
df

,StudyHours,SleepHours,CoffeeCups,Score,Pass
0,1,9,0,3,0
1,2,8,1,5,0
2,3,7,1,8,1
3,4,7,2,12,1
4,5,6,2,15,1
5,6,6,3,14,1
6,7,5,3,10,1
7,8,5,4,7,0
8,9,4,4,4,0
9,10,4,5,2,0


In [202]:
X = df.drop(columns=["Pass", "Score"], axis=1)
y = df["Pass"]
X, y

(   StudyHours  SleepHours  CoffeeCups
 0           1           9           0
 1           2           8           1
 2           3           7           1
 3           4           7           2
 4           5           6           2
 5           6           6           3
 6           7           5           3
 7           8           5           4
 8           9           4           4
 9          10           4           5,
 0    0
 1    0
 2    1
 3    1
 4    1
 5    1
 6    1
 7    0
 8    0
 9    0
 Name: Pass, dtype: int64)

In [203]:
tree = ClassificationTree()

In [204]:
tree.fit(X,y)

{'type': 'node',
 'feature': 'StudyHours',
 'threshold': np.float64(7.5),
 'left': {'type': 'node',
  'feature': 'StudyHours',
  'threshold': np.float64(2.5),
  'left': {'type': 'leaf', 'prediction': np.int64(0), 'depth': 2},
  'right': {'type': 'node',
   'feature': 'StudyHours',
   'threshold': np.float64(3.5),
   'left': {'type': 'leaf', 'prediction': np.int64(1), 'depth': 3},
   'right': {'type': 'leaf', 'prediction': np.int64(1), 'depth': 3},
   'depth': 2},
  'depth': 1},
 'right': {'type': 'leaf', 'prediction': np.int64(0), 'depth': 1},
 'depth': 0}

In [205]:
tree.predict({"CoffeeCups": 3, "SleepHours": 7, "StudyHours": 10})

np.int64(0)